In [1]:
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer  # Explicitly enable experimental features
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import defaultdict

from dhs_modelling_functions import final_ds_droping_cols, fold_generator
import sys
print(sys.executable)

/mnt/datadisk/sharedPrograms/venv/water_2202_tf82/bin/python3


In [2]:
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
urban_rural_all_mode = 'all' # 'all', 'U', 'R'
group_by_col = 'adm2_gaul'


# Scale options
scale_all_data = False
# leave_out_encodings = True
# zero_one_scale_categorical = False
# scale_labels = True
scale_numerical_data = True

# More Options
drop_agriculture = False
fold_by = 'survey'
mask_categoricals_together = False  # Note categorical values will be missing together
split_categoricals = False  # Note categorical values will be missing together
drop_perc = 19

test_mode = True


in_f = f"{input_dir}5_grouped_df_V3_{dataset_type}_{group_by_col}_joined_with_ipc_{urban_rural_all_mode}.pkl"
out_f = f"{input_dir}imputation_results_fold_by_{fold_by}_mask_categoricals_together_{mask_categoricals_together}_split_categoricals_{split_categoricals}_drop_perc_{drop_perc}.csv"

In [3]:
input_df = pd.read_pickle(in_f)

for c in input_df.columns:
    if 'year' in c or 'Year' in c:
        print(c)

# if urban_rural_all_mode == 'U':
#     drop_agriculture = True
drop_datasets = ['FS', 'Meta one-hot encoding', 'Meta frequency encoding']
    
df = final_ds_droping_cols(input_df, drop_meta=True, drop_food_help=True, drop_perc=drop_perc,
                           retain_month=False, drop_highly_correlated_cols=False, drop_region=True, 
                 drop_data_sets=drop_datasets, 
                 use_NAN_amount_and_replace_NANs_in_categorical=False, drop_agricultural_cols=drop_agriculture, 
                 drop_below_version=False, numerical_data=['mean'], retain_adm=False, 
                 retain_GEID_init=False, numerical_return_format='float64', verbose=3)


drop_cols = [c for c in df.columns if 'FS;' in c and not '0-2y' in c]
df = df.drop(columns=drop_cols)

for col in df.columns:
    print(col)

df

Meta; year
DHS Cat; person fetching water: female child under 15 years old
DHS Cat; person fetching water: male child under 15 years old
Dropped FS; data: ['FS; IPC: 0-5m: mean', 'FS; IPC: 0-5m: max', 'FS; IPC: 0-5m: min', 'FS; IPC: 0-5m: median', 'FS; IPC + FH: 0-5m: mean', 'FS; IPC + FH: 0-5m: max', 'FS; IPC + FH: 0-5m: min', 'FS; IPC + FH: 0-5m: median', 'FS; IPC: 6-12m: mean', 'FS; IPC: 6-12m: max', 'FS; IPC: 6-12m: min', 'FS; IPC: 6-12m: median', 'FS; IPC + FH: 6-12m: mean', 'FS; IPC + FH: 6-12m: max', 'FS; IPC + FH: 6-12m: min', 'FS; IPC + FH: 6-12m: median', 'FS; IPC: 1-2y: mean', 'FS; IPC: 1-2y: max', 'FS; IPC: 1-2y: min', 'FS; IPC: 1-2y: median', 'FS; IPC + FH: 1-2y: mean', 'FS; IPC + FH: 1-2y: max', 'FS; IPC + FH: 1-2y: min', 'FS; IPC + FH: 1-2y: median', 'FS; IPC: 2-4y: mean', 'FS; IPC: 2-4y: max', 'FS; IPC: 2-4y: min', 'FS; IPC: 2-4y: median', 'FS; IPC + FH: 2-4y: mean', 'FS; IPC + FH: 2-4y: max', 'FS; IPC + FH: 2-4y: min', 'FS; IPC + FH: 2-4y: median', 'FS; IPC: 4-6y: mean

,Meta; GEID_init,Meta; adm0_gaul,Meta; year,DHS Num; number of household members: mean,DHS Num; owns sheep: mean,DHS Num; number of eligible children for height and weight: mean,DHS Num; total adults measured: mean,DHS Num; number of mosquito bed nets: mean,DHS Num; number of eligible children for height & weight: mean,DHS Num; cluster altitude in meters: mean,...,DHS Cat; water usually treated by: use water filter: yes,"DHS Cat; owns livestock, herds or farm animals: yes",DHS Cat; number of households sharing toilet: applicable: applicable,DHS Cat; URBAN_RURA: R,DHS Sustainbench; asset_index,DHS Sustainbench; water_index,DHS Sustainbench; sanitation_index,DHS Sustainbench; under5_mort,DHS Sustainbench; women_edu,DHS Sustainbench; women_bmi
0,AOGE52FL,Angola,2007.0,5.160000,NaN,NaN,1.040000,NaN,1.400000,NaN,...,NaN,NaN,0.480000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,AOGE52FL,Angola,2006.0,5.280000,NaN,NaN,1.160000,NaN,1.240000,NaN,...,NaN,NaN,0.560000,1.000000,NaN,NaN,NaN,32.258065,3.777778,NaN
2,AOGE52FL,Angola,2007.0,4.000000,NaN,NaN,1.040000,NaN,0.600000,NaN,...,NaN,NaN,0.760000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,AOGE52FL,Angola,2006.0,5.208333,NaN,NaN,1.250000,NaN,0.875000,NaN,...,NaN,NaN,0.125000,1.000000,NaN,NaN,NaN,43.478261,2.142857,NaN
4,AOGE52FL,Angola,2006.0,4.210526,NaN,NaN,0.789474,NaN,1.157895,NaN,...,NaN,NaN,0.210526,1.000000,NaN,NaN,NaN,45.454545,1.500000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15247,ZWGE72FL,Zimbabwe,2015.0,4.642857,0.297619,0.666667,1.797619,0.285714,NaN,1229.666667,...,0.02381,0.809524,0.750000,1.000000,-0.453354,3.190476,2.702381,25.641026,9.152542,23.489245
15248,ZWGE72FL,Zimbabwe,2015.0,4.075949,0.341772,0.632911,1.556962,0.924051,NaN,988.126582,...,0.00000,0.848101,0.569620,1.000000,-0.760985,3.253165,2.202532,37.037037,7.830508,22.874510
15249,ZWGE72FL,Zimbabwe,2015.0,4.366337,0.376238,0.673267,1.554455,0.702970,NaN,773.574257,...,0.00000,0.930693,0.277228,1.000000,-1.155604,3.415842,1.633663,12.987013,8.284211,22.965455
15250,ZWGE72FL,Zimbabwe,2015.0,4.546154,0.107692,0.723077,2.030769,1.676923,NaN,1276.853846,...,0.00000,0.638462,0.853846,1.000000,-0.473785,3.523077,2.515385,0.000000,8.313559,23.995091


In [4]:
def calc_means_of_columns_of_multiple_dfs(df_list):
    # Concatenate all DataFrames in imputed_test_datasets
    concat_df = pd.concat(df_list, keys=range(len(df_list)))

    # Reset the index and rename the columns to make them easier to work with
    concat_df.reset_index(inplace=True)
    concat_df.rename(columns={'level_0': 'df_index', 'level_1': 'row_index'}, inplace=True)

    # Group by the row_index and calculate the mean of each group
    summary_df = concat_df.groupby('row_index').mean()

    # Drop the df_index column, as it's no longer needed
    summary_df.drop(columns='df_index', inplace=True)
    return summary_df

In [5]:
# Define a list of imputers to test
imputers = [
    # {"name": "MICE BayesianRidge", "imputer": IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0)},
    # {"name": "MICE LinearRegression", "imputer": IterativeImputer(estimator=LinearRegression(), max_iter=10, random_state=0)},
    # {"name": "MICE RandomForest", "imputer": IterativeImputer(estimator=RandomForestRegressor(n_estimators=10), max_iter=10, random_state=0)},
    # {"name": "MICE KNN", "imputer": IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=5), max_iter=10, random_state=0)},
    # # {"name": "BayesianRidge", "imputer": BayesianRidge()},
    # {"name": "LinearRegression", "imputer": LinearRegression()},
    {"name": "KNN", "imputer": KNNImputer(n_neighbors=5)},
    # {"name": "RandomForest", "imputer": RandomForestRegressor(n_estimators=10)},
    # {"name": "SimpleImputer", "imputer": SimpleImputer(strategy="mean")},
]

missingness_fraction = 0.2  # Fraction of values to be made missing in the test set

numerical_data = df.select_dtypes(include=[np.number])
numerical_data_cols = numerical_data.columns

# Scale data
if scale_all_data:
    scaler = StandardScaler()
    numerical_data_s = scaler.fit_transform(numerical_data)
    numerical_data = pd.DataFrame(numerical_data_s, index=numerical_data.index, columns=numerical_data.columns)
elif scale_numerical_data:
    numerical_cols = [c for c in numerical_data_cols if 'DHS Num;' in c]
    scaler = StandardScaler()
    numerical_data_s = scaler.fit_transform(numerical_data[numerical_cols])
    numerical_data_ds = pd.DataFrame(numerical_data_s, index=numerical_data.index, columns=numerical_cols)
    numerical_data[numerical_cols] = numerical_data_ds
    

fold_gen = fold_generator(df, fold_by, n_splits=5)

# find columns which depend on each other
base_cols = {}
found_base_col = {}
for col in numerical_data.columns:
    # base_col is the part of the column name before the last colon
    col_split = col.rsplit(':', 1)
    if col_split[-1] == ' applicable':
        col_split = col_split[0].rsplit(':', 1)
    base_col = col_split[0]
    if base_col not in base_cols:
        cols = [c for c in numerical_data.columns if c.startswith(base_col)]
        base_cols[base_col] = cols
        for c in cols:
            found_base_col[c] = base_col

# Maximum length of base_cols
max_base_cols = max([len(v) for v in base_cols.values()])

# Initialize a dictionary to store the imputation results for each fold and imputer
imputation_results = defaultdict(dict)

# Apply and store imputations for each imputer and each column
for imputer_info in imputers:
    imputer_name = imputer_info["name"]
    print(imputer_name)
    imputer = imputer_info["imputer"]
    
    fold_results = {}
    complete_imputed_ds = {}
    # Iterate over each fold
    for fold, (train_index, test_index) in enumerate(fold_gen):
        print(fold)
        X_train, X_test = numerical_data.iloc[train_index], numerical_data.iloc[test_index]
        
        # Introduce artificial missing values in a copy of X_test
        X_test_with_missing = X_test.copy()
        missing_mask = np.zeros(X_test_with_missing.shape, dtype=bool)  # Initialize the mask

        if mask_categoricals_together:
            processed_base_cols = set()
            # Introduce missing values in the test set
            for col in X_test_with_missing.columns:
                if col in processed_base_cols:
                    continue
                non_missing_indices = X_test_with_missing[col].dropna().index
                random_sample = np.random.choice(non_missing_indices, 
                                                size=int(np.ceil(missingness_fraction * len(non_missing_indices))), 
                                                replace=False)
                local_indices = X_test_with_missing.index.get_indexer(random_sample)
                
                if col in found_base_col:
                    for col2 in base_cols[found_base_col[col]]:
                        processed_base_cols.add(col2)
                        missing_mask[local_indices, X_test_with_missing.columns.get_loc(col2)] = True
                        X_test_with_missing.loc[random_sample, col2] = np.nan
                else:
                    missing_mask[local_indices, X_test_with_missing.columns.get_loc(col)] = True
                    X_test_with_missing.loc[random_sample, col] = np.nan
        else:       
            for col in X_test_with_missing.columns:
                non_missing_indices = X_test_with_missing[col].dropna().index
                random_sample = np.random.choice(non_missing_indices, 
                                                size=int(np.ceil(missingness_fraction * len(non_missing_indices))), 
                                                replace=False)
                local_indices = X_test_with_missing.index.get_indexer(random_sample)
                missing_mask[local_indices, X_test_with_missing.columns.get_loc(col)] = True
                X_test_with_missing.loc[random_sample, col] = np.nan
                

        cols_to_use_l = []
        imputed_numericals_datasets = []
        imputed_test_datasets = []
        
        if split_categoricals:
            # Drop categorical columns of the same category to disallow imputation of one column based on the same category (which will be missing together in the test set)
            for i in range(max_base_cols):
                cols_to_drop = []
                for cols in base_cols.values():
                    cols = cols.copy()
                    if i > len(cols) - 1:
                        # iterate over columns if i is greater than the number of columns
                        j = i % len(cols)
                    else:
                        j = i

                    # drop jth column
                    removed_col = cols.pop(j)
                    cols_to_drop.extend(cols)
                # print('1', cols_to_drop)  
                cols_to_use = X_train.columns.difference(cols_to_drop)
                # print('2', cols_to_use)                  
                cols_to_use_l.append(cols_to_use)
        else:
            cols_to_use_l.append(X_train.columns)
        
        for nr, cols_to_use in enumerate(cols_to_use_l):
            # print(cols_to_use)
            X_train_local = X_train[cols_to_use]
            X_test_with_missing_local = X_test_with_missing[cols_to_use]
            numerical_data_local = numerical_data[cols_to_use]

            # Fit the imputer
            imputer.fit(X_train_local)
            # Predict the missing values in the test set
            X_test_imputed = pd.DataFrame(imputer.transform(X_test_with_missing_local), 
                                          columns=X_test_with_missing_local.columns, index=X_test_with_missing_local.index)
            imputed_test_datasets.append(X_test_imputed)
            
            # Predict all data
            numerical_data_imputed = pd.DataFrame(imputer.transform(numerical_data_local),
                                                    columns=numerical_data_local.columns, index=numerical_data_local.index)
            imputed_numericals_datasets.append(numerical_data_imputed)
            # # Fill the imputed values back to the test set copy
            # for column in cols_to_use:
            #     X_test_with_missing_local[column] = X_test_imputed[:, X_test_with_missing_local.columns.get_loc(column)]
            #     numerical_data_local[column] = numerical_data_imputed[:, numerical_data_local.columns.get_loc(column)]
            
            # print(X_test_imputed.head())
        
        if split_categoricals:
            # calculate means of imputed datasets
            num_imputed = calc_means_of_columns_of_multiple_dfs(imputed_numericals_datasets)
            X_test_imputed = calc_means_of_columns_of_multiple_dfs(imputed_test_datasets)
            print(X_test_imputed.shape, X_test.shape, num_imputed.shape, numerical_data.shape)
            # print(num_imputed.head(20))
        else:
            num_imputed = imputed_numericals_datasets[0]
            X_test_imputed = imputed_test_datasets[0]
        
        # Store the imputation results and masks for each fold
        imputation_results[imputer_name][fold] = {
            "X_test": X_test,
            "X_test_with_missing": X_test_with_missing,
            "X_test_imputed": X_test_imputed,
            "complete_imputed_ds": num_imputed,
            "missing_mask": missing_mask,
        }
        
        if test_mode:
            break


MICE KNN
0


In [6]:
# Evaluation phase
comparison_results = {}
for imputer_name, imputer_info in imputation_results.items():
    comparison_results[imputer_name] = {}

    all_original_values = defaultdict(list)
    all_imputed_values = defaultdict(list)
    for fold, results in imputer_info.items():
        X_test = results["X_test"]
        missing_mask = results["missing_mask"]
        X_test_imputed = results["X_test_imputed"]
        all_imputed = results["complete_imputed_ds"]
        all_imputed_t = scaler.inverse_transform(all_imputed)
        all_imputed_t = pd.DataFrame(all_imputed_t, index=numerical_data.index, columns=numerical_data.columns)
        all_imputed = pd.concat([all_imputed_t, df.drop(columns=numerical_data.columns)], axis=1)
        all_imputed.to_csv(f"{out_f[:-4]}_{imputer_name}_DF.csv")
        
        assert all_imputed_t.index.equals(df.index)
        df_imputed = df.copy()
        for col in all_imputed_t.columns:
            df_imputed[col].fillna(all_imputed_t[col], inplace=True)
        # df_imputed.to_csv(f"{out_f[:-4]}{imputer_name}_replace_nans.csv")
        df_imputed.to_csv(f"{out_f[:-4]}_{imputer_name}_DF_imputed.csv")

        
        # Evaluate and store the metrics
        for col in X_test.columns:
            mask = missing_mask[:, X_test.columns.get_loc(col)]
            original_values = X_test.loc[mask, col]
            imputed_values = X_test_imputed.loc[mask, col]
            all_original_values[col].extend(original_values)
            all_imputed_values[col].extend(imputed_values)

            mse = mean_squared_error(original_values, imputed_values)
            # correlation = np.corrcoef(original_values, imputed_values)[0, 1] if len(original_values) > 1 else 0
            r2 = r2_score(original_values, imputed_values)
            if len(original_values) > 1 and np.std(original_values) != 0 and np.std(imputed_values) != 0:
                correlation = np.corrcoef(original_values, imputed_values)[0, 1]
            else:
                print(f"Warning: correlation could not be calculated for {imputer_name} and column {col} in fold {fold}")
                print(f"Original values: {original_values}")
                print(f"Imputed values: {imputed_values}")
                correlation = np.nan

            comparison_results[imputer_name].setdefault(col, {}).update({
                f"RMSE: Fold {fold + 1}": np.sqrt(mse),
                f"Correlation: Fold {fold+1}": correlation,
                f"R2: Fold {fold+1}": r2
            })
    
    for col in X_test.columns:
        # available data in original df
        available_data = len(df[col].dropna())/len(df)
        # available data in original df and not 0
        if 'DHS Cat;' in col:
            non_zero_data = len(df[df[col] != 0][col].dropna())/len(df)
        else:
            non_zero_data = available_data
        all_original_values_col = np.array(all_original_values[col])
        all_imputed_values_col = np.array(all_imputed_values[col])
        mse = mean_squared_error(all_original_values_col, all_imputed_values_col)
        if len(all_original_values_col) > 1 and np.std(all_original_values_col) != 0 and np.std(all_imputed_values_col) != 0:
            correlation = np.corrcoef(all_original_values_col, all_imputed_values_col)[0, 1]
        else:
            print(f"Warning2: correlation could not be calculated for {imputer_name} and column {col} in fold {fold}")
            print(f"Original values: {all_original_values_col}")
            print(f"Imputed values: {all_imputed_values_col}")
            correlation = np.nan
        # correlation = np.corrcoef(all_original_values_col, all_imputed_values_col)[0, 1] if len(all_original_values_col) > 1 else 0
        r2 = r2_score(all_original_values_col, all_imputed_values_col)
        comparison_results[imputer_name][col].update({
            "RMSE overall": np.sqrt(mse),
            "Correlation overall": correlation,
            "R2 overall": r2,
            f"Available data": available_data,
            f"Available and non-zero distribution data": non_zero_data,
            # "RMSE weighted by available data": f"{np.sqrt(mse) * available_data:.2f}",
            # "Correlation weighted by available data": correlation * available_data,
        })

# Convert comparison_results to a DataFrame
data_to_append = []
for imputer_name, results in comparison_results.items():
    for fold_col, metrics in results.items():
        data_to_append.append({
            "Imputer": imputer_name,
            "Column": fold_col,
            **metrics
        })
        

ValueError: operands could not be broadcast together with shapes (15252,139) (18,) (15252,139) 

In [ ]:
result_df = pd.DataFrame(data_to_append)
result_df = result_df.set_index(["Imputer", "Column"])
print(result_df)
# result_df.to_csv(out_f)
# result_df.to_csv(f"{out_f[:-4]}_maskcats{mask_categoricals_together}_splitcats{split_categoricals}.csv")

all_result_dfs = []
# Calculate mean and standard deviation for each metric and each Imputer
groupy_by_imputer = result_df.groupby("Imputer")
mean_df = groupy_by_imputer.mean()
mean_df["Column"] = "Mean"
std_df = groupy_by_imputer.std()
std_df["Column"] = "Std"
all_result_dfs.extend([mean_df, std_df])

#Calculate means, std, weighted means and weighted std for grouped objects an by available data
for i in range(100, drop_perc - 10, -10):
    result_sub_df = result_df[(result_df["Available data"] >= i/100) & (result_df["Available data"] < (i+10)/100)]
    groupy_by_imputer = result_sub_df.groupby("Imputer")
    mean_df = groupy_by_imputer.mean()
    
    mean_df["Column"] = f"Mean available data {i}-{i+10}"
    std_df = groupy_by_imputer.std()
    std_df["Column"] = f"Std available data {i}-{i+10}"
    all_result_dfs.extend([mean_df, std_df])

groupy_by_imputer = result_df.groupby("Imputer")
for av_d in ["Available data", "Available and non-zero distribution data"][:1]:
    avg_mean_df = groupy_by_imputer.apply(lambda x: pd.Series(np.average(x, weights=x[av_d], axis=0), index=x.columns))
    weighted_std_df = groupy_by_imputer.apply(lambda x: pd.Series(np.average((x - avg_mean_df)**2, weights=x[av_d], axis=0), index=x.columns))
    avg_mean_df["Column"] = f"Weighted mean: {av_d}"
    weighted_std_df["Column"] = f"Weighted std: {av_d}"
    all_result_dfs.extend([avg_mean_df, weighted_std_df])

all_result_dfs.append(result_df)


In [ ]:
mean_df

In [ ]:
std_df = groupy_by_imputer.std()
avg_mean_df["Column"] = "Mean weighted by available data"

std_df["Column"] = "Standard deviation"
# print(mean_df)
# print(result_df)
result_df = pd.concat([df.reset_index() for df in all_result_dfs], axis=0)
result_df.to_csv(out_f)
result_df[['Imputer', 'Column', 'RMSE overall', 'Correlation overall', 'R2 overall']]